# Stiffness ration for non linear equation

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# imports
import torch
import matplotlib.pyplot as plt
import numpy as np
from scipy.integrate import solve_ivp
from numpy import linalg as LA

In [6]:
# x_range = [0., 1.]
# beta = 0.9

# def Exemple1(t, y, alpha, beta=beta):
#     if isinstance(y, np.ndarray):
#         yp = np.ones_like(y)
#         yp[0] = -alpha*(y[0] + beta*y[1]**2)
#         yp[1] = y[0] - beta*y[1]**2 + 1
#     else:
#         yp = torch.ones_like(y)
#         yp[:, 0, :]  = -alpha*y[:, 0, :] + beta*y[:, 1, :]
#         yp[:, 1, :] = y[:, 0, :] - beta*y[:, 1, :]**2 + 1
#     return yp

# IC = torch.tensor([[2.], [0.0]])
# J_function = lambda x, y, alpha, beta=beta: np.matrix([[-alpha, 2*beta*y], [1, -2*beta*y]])

# alpha_list = [i*2 for i in [2, 4, 8, 16, 32, 64, 128, 256, 512]]
# equation_list = [lambda t, y, i=i: Exemple1(t, y, alpha=i) for i in alpha_list]

In [7]:
x_range = [0., 1.]
beta = 0.1
a = 2

def Exemple2(t, y, alpha, beta=beta, a=a):
    if isinstance(y, np.ndarray):
        yp = np.ones_like(y)
        yp[0] = y[1]
        yp[1] = -a*beta*alpha*(y[1]**2-y[0]**2) - alpha*y[1]
    else:
        yp = torch.ones_like(y)
        yp[:, 0, :] = y[:, 1, :]
        yp[:, 1, :] = -a*beta*alpha*(y[:, 1, :]**2-y[:, 0, :]**2) - alpha*y[:, 1, :]
    return yp

IC = torch.tensor([[2.], [0.0]])
J_function = lambda x, y, alpha, beta=beta, a=a: np.matrix([[0, 1], [2*a*beta*alpha*x, -2*a*beta*alpha*y]])

alpha_list = [i for i in [2, 4, 8, 16, 32, 64, 128, 256, 512]]
equation_list = [lambda t, y, i=i: Exemple2(t, y, alpha=i) for i in alpha_list]

In [8]:
# # VAN DER POLS
# x_range = [0., 100.]

# def VanDerPols(t, y, alpha):
#     if isinstance(y, np.ndarray):
#         yp = np.ones_like(y)
#         yp[0] = y[1]
#         yp[1] = alpha*(1-y[0]**2)*y[1] - y[0]
#     else:
#         yp = torch.ones_like(y)
#         yp[:, 0, :]  = y[:, 1, :]
#         yp[:, 1, :] = alpha*(1-y[:, 0, :]**2)*y[:, 1, :] - y[:, 0, :]
#     return yp

# IC = torch.tensor([[2.], [0.]])
# J_function = lambda x, y, alpha: np.matrix([[0, 1], [(1-x**2)*alpha, -2*alpha*x*y-1]])

# alpha_list = [2, 4, 8, 16, 32, 64, 128, 256, 516]
# #alpha_list = [516]
# equation_list = [lambda t, y, i=i: VanDerPols(t, y, alpha=i) for i in alpha_list]

In [9]:
t_grid = torch.linspace(x_range[0], x_range[1], 10000)

In [10]:
fig, ax = plt.subplots(figsize=(15, 6))
ax.semilogy()  
ax.set_ylabel("Stiffness Ratio")
ax.set_xlabel("t")
ax.set_title("Stiffness Ratio for various stiffness parameter alpha vs t")

is_complex = lambda c: True if isinstance(c, complex) else False
for alpha, equation in zip(alpha_list, equation_list):
   true_radau = lambda x, v: (solve_ivp(equation, [x_range[0], x_range[1]],
                                     v.squeeze(), t_eval=x.squeeze(),
                                     method="Radau").y)
   true_solution = true_radau(t_grid, IC)
   stiffness_ratio = []
   for t, x, y in zip(t_grid, true_solution[0, :], true_solution[1, :]):
      J = J_function(x, y, alpha)
      eigenvalues, eigenvectors = LA.eig(J)
      eigenvalues_abs = np.zeros(2)
      for i, eigenvalue in enumerate(eigenvalues):
         if is_complex(eigenvalue):
            eigenvalues_abs[i] = np.abs(eigenvalue.real)
         else:
            eigenvalues_abs[i] = np.abs(eigenvalue)
      min_eigenvalues = np.min(eigenvalues_abs)
      max_eigenvalues = np.max(eigenvalues_abs)
      stiffness_ratio.append(max_eigenvalues/min_eigenvalues)
   ax.plot(t_grid[1:], stiffness_ratio[1:], label=f"alpha={alpha}")
ax.legend(loc="best")